<a href="https://colab.research.google.com/github/manavmittal05/Large-Language-Models/blob/main/A2/Q1_ReAct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers huggingface_hub
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes
!pip install accelerate
!pip install datasets
!pip install langchain langchain_core langchain_community

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-q5stqlb0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-q5stqlb0
^C
ERROR: Operation cancelled by user
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-v7tbbl3r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-v7tbbl3r
  Resolved https://github.com/huggingface/accelerate.git to commit 4305033f8035defad0a87cd38e5c918e78510ba5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.35.0.dev0-py3-none-any.whl size=330627 sha256=f6e4b5d2bd9f324318b75b4211cc9e8be5c1ebdd89445799f72010e2801fa5ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-tfuisg1e/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35

In [4]:
from langchain_core.language_models.llms import LLM
import transformers
import torch
import pandas as pd
from pydantic import Field
from typing import Any, Dict

In [5]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent


In [6]:
from huggingface_hub import login

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

login(secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
class CustomLLM(LLM):

	pipeline: Any

	def __init__(self, pipeline:Any):

		super().__init__(pipeline=pipeline)

		self.pipeline = pipeline




	def _call(self, prompt, **kwargs: Dict[str, Any]):

		messages = [
			{"role": "user", "content": prompt},
		]

		outputs = self.pipeline(
			messages,
			max_new_tokens=512
		)
		return outputs[0]["generated_text"][-1]["content"]

	@property
	def _llm_type(self) -> str:
		"""Get the type of language model used by this chat model. Used for logging purposes only."""
		return "CustomLLM"


In [49]:
class Dataset:

    def __init__(self):
        self.dataset = pd.read_parquet("hf://datasets/cais/mmlu/college_mathematics/test-00000-of-00001.parquet")


    def get_react_prompt(self, idx):
        question = self.dataset.iloc[idx]['question']
        options = self.dataset.iloc[idx]['choices']

        prompt = f"""
        
        Answer the following questions as best you can.

        Use the following format:

        Question: the input question you must answer
        Thought: you should always think about what to do
        Action: the action to take
        Action Input: the input to the action
        Observation: the result of the action
        ... (this Thought/Action/Action Input/Observation can repeat N times)
        Thought: I now know the final answer
        Final Answer: the final answer to the original input question


        Choose the answer to the given question from below options. I dont need explanation.
        Keep the Answer in format "Option [x]", where x is 1,2,3,4.

        {question}

        Option 1: {options[0]}
        Option 2: {options[1]}
        Option 3: {options[2]}
        Option 4: {options[3]}

        Answer:"""

        return prompt

In [50]:
dataset = Dataset()


In [10]:
pipeline_llama = transformers.pipeline(
			"text-generation",
			model="meta-llama/Meta-Llama-3.1-8B-Instruct",
			device_map="auto",
			model_kwargs={
			"torch_dtype": torch.bfloat16,
			"quantization_config": {"load_in_4bit": True},
			"low_cpu_mem_usage": True,
			}
		)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [11]:
pipeline_phi = transformers.pipeline(
			"text-generation",
			model="microsoft/Phi-3.5-mini-instruct",
			device_map="auto",
			model_kwargs={
			"torch_dtype": torch.bfloat16,
			"quantization_config": {"load_in_4bit": True},
			"low_cpu_mem_usage": True,
			}
		)

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [33]:
pipeline_gemma = transformers.pipeline(
			"text-generation",
			model="google/gemma-2b-it",
			device_map="auto",
            max_new_tokens=1024,
			model_kwargs={
			"torch_dtype": torch.bfloat16,
			"quantization_config": {"load_in_4bit": True},
			"low_cpu_mem_usage": True,
			}
		)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
llama = CustomLLM(pipeline=pipeline_llama)

In [14]:
phi = CustomLLM(pipeline=pipeline_phi)

In [34]:
gemma = CustomLLM(pipeline=pipeline_gemma)

In [16]:
from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser

In [18]:
tools_llama = load_tools(["llm-math"], llama)
agent_llama = initialize_agent(tools_llama, llama, agent="zero-shot-react-description",
                         handle_parsing_errors = True,
                         agent_executor_kwargs={ "output_parser": ReActSingleInputOutputParser},
                         max_iterations=1, return_intermediate_steps=True)

/tmp/ipykernel_36/1577576976.py:2: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent_llama = initialize_agent(tools_llama, llama, agent="zero-shot-react-description",


In [19]:
tools_phi = load_tools(["llm-math"], phi)
agent_phi = initialize_agent(tools_phi, phi, agent="zero-shot-react-description",
                         handle_parsing_errors = True,
                         agent_executor_kwargs={ "output_parser": ReActSingleInputOutputParser},
                         max_iterations=1, return_intermediate_steps=True)

In [35]:
tools_gemma = load_tools(["llm-math"], gemma)
agent_gemma = initialize_agent(tools_gemma, gemma, agent="zero-shot-react-description",
                         handle_parsing_errors = True,
                         agent_executor_kwargs={ "output_parser": ReActSingleInputOutputParser},
                         max_iterations=1, return_intermediate_steps=True)

In [21]:
from tqdm import tqdm

In [ ]:
df = pd.DataFrame(columns=['gemma', 'phi', 'llama'])

for i in tqdm(range(len(dataset.dataset))):
    react_prompt = dataset.get_react_prompt(i)
    
    gemma_out = agent_gemma({'input': react_prompt})['intermediate_steps']
    gemma_answer = ""
    if len(gemma_out) == 1:
        gemma_answer = gemma_out[0][0].log
        
    phi_out = agent_phi({'input': react_prompt})['intermediate_steps']
    phi_answer = ""
    if len(phi_out) == 1:
        phi_answer = phi_out[0][0].log
        
    llama_out = agent_llama({'input': react_prompt})['intermediate_steps']
    llama_answer = ""
    if len(llama_out) == 1:
        llama_answer = llama_out[0][0].log
        

    new_row = {'gemma': gemma_answer, 'phi': phi_answer, 'llama': llama_answer}
    df.loc[len(df)] = new_row

# save the chain-of-thought answers in a csv file
df.to_csv('react_answers.csv', index=False)

  5%|▌         | 5/100 [05:58<1:53:41, 71.81s/it]